# Cartoonify faces

## Apply Video processing and Face detection techniques to place a cartoon mask instead of your face in the video.

In [25]:
import cv2
import os

# Load the facemask image and the video stream
facemask = cv2.imread('cartoon.png', -1)
cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Initialize the face detection algorithm
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Read each frame of the video stream
    ret, frame = cap.read()
    
    # Apply the face detection algorithm to detect the faces in the frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    # For each detected face, resize the facemask image and apply the facemask on the face
    for (x, y, w, h) in faces:
        resized_facemask = cv2.resize(facemask, (w, h))
        alpha_mask = resized_facemask[:, :, 3] / 255.0
        alpha_image = 1.0 - alpha_mask
        for c in range(0, 3):
            frame[y:y+h, x:x+w, c] = (alpha_mask * resized_facemask[:, :, c] + alpha_image * frame[y:y+h, x:x+w, c])
    
    # Display the resulting frame with the facemask overlay
    cv2.imshow('Facemask Overlay', frame)
    
    # Write the frame to the output video
    out.write(frame)
    
    # Wait for a small amount of time to allow the frame to be displayed on the screen
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video stream and close all windows
cap.release()
out.release()
cv2.destroyAllWindows()